In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
from datasets import Dataset, load_metric
import mmh3
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification
import torch
import random

In [3]:
MAX_LEN_CHARS = 256*3

In [4]:
df_t = pd.read_csv("data/eng_sentences.tsv", sep="\t", names=["id", "lang", "text"])
df_t = df_t[df_t["text"].str.len() < MAX_LEN_CHARS]

# shuffle
df_t["id"] = df_t["text"].map(lambda x: mmh3.hash64(x.encode('utf8'))[0])
df_t = df_t.sort_values("id")
len(df_t)

1582094

In [5]:
df_o = pd.read_csv("data/oss.tsv", sep="\t", names=["text"])
df_o = df_o[df_o["text"].str.len() < MAX_LEN_CHARS]

# shuffle
df_o["id"] = df_o["text"].map(lambda x: mmh3.hash64(x.encode('utf8'))[0])
df_o = df_o.sort_values("id")
len(df_o)

66939

In [6]:
df_g = pd.read_csv("data/gutenberg.tsv", sep="\t", names=["text"])
df_g = df_g[df_g["text"].str.len() < MAX_LEN_CHARS]

# shuffle
df_g["id"] = df_g["text"].map(lambda x: mmh3.hash64(x.encode('utf8'))[0])
df_g = df_g.sort_values("id")
len(df_g)

4102516

In [7]:
df_t.head(1)

,id,lang,text
1123557,-9223326714129212706,eng,Tom told everybody that he didn't know what to...


In [8]:
df_o.head(1)

,text,id
12494,Sometimes you need to access a type passed as ...,-9222670596738368818


In [9]:
df_g.head(1)

,text,id
1285889,To persons standing alone on a hill during a c...,-9223366647700897551


In [12]:
train_t_size = 1000000
train_o_size = 66939-20000
train_g_size = 1000000
val_size = 10000
test_size = 10000
assert(train_t_size + val_size + test_size <= len(df_t["text"]))
assert(train_o_size + val_size + test_size <= len(df_o["text"]))
assert(train_g_size + val_size + test_size <= len(df_g["text"]))

train_text = list(df_t["text"])[:train_t_size] + list(df_o["text"])[:train_o_size] + list(df_g["text"])[:train_g_size]
random.Random(4).shuffle(train_text)

val_text = list(df_t["text"])[-val_size-test_size:-test_size] + list(df_o["text"])[-val_size-test_size:-test_size] \
+ list(df_g["text"])[-val_size-test_size:-test_size]

test_text = list(df_t["text"])[-test_size:] + list(df_o["text"])[-test_size:] + list(df_g["text"])[-test_size:]

len(train_text), len(val_text), len(test_text)

(2046939, 30000, 30000)

In [13]:
train_text[:3]

["And with my sister-nymphs I sport, Till the broad sun looks o'er the floods; Then, swift we seek our crystal court, Deep in the wave, 'mid Neptune's woods.",
 '"To learn to know nothing, go whither you are ignorant.',
 "He's very angry with her."]

In [14]:
label_list = ['O', 'B-COMMA']
label_encoding_dict = {'O': 0, 'B-COMMA': 1}

model_checkpoint = "distilroberta-base"
batch_size = 128

In [15]:
if "roberta" in model_checkpoint:
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)
else:
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))
args = TrainingArguments(
    "comma-" + model_checkpoint,
    evaluation_strategy="steps",
    eval_steps=500,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=1e-5,
)

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForTokenClassification: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream tas

In [16]:
def to_dataset(texts):
    tokens, tags = [], []
    for text in texts:
        text_tokens, text_tags = [], []
        for token in text.split(" "):
            if "," in token:
                tag = "B-COMMA"
                token = token.replace(",", "")
            else:
                tag = "O"
            text_tokens.append(token)
            text_tags.append(tag)
        tokens.append(text_tokens)
        tags.append(text_tags)
    return Dataset.from_pandas(pd.DataFrame({
        'tokens': tokens, 'tags': tags
    }))

In [17]:
train_dataset = to_dataset(train_text)
val_dataset = to_dataset(val_text)
test_dataset = to_dataset(test_text)

train_dataset

Dataset({
    features: ['tokens', 'tags'],
    num_rows: 2046939
})

In [18]:
def tokenize_and_align_labels(examples):
    label_all_tokens = True
    tokenized_inputs = tokenizer(list(examples["tokens"]), truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples["tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif label[word_idx] == '0':
                label_ids.append(0)
            elif word_idx != previous_word_idx:
                label_ids.append(label_encoding_dict[label[word_idx]])
            else:
                label_ids.append(label_encoding_dict[label[word_idx]] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
        
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

train_tokenized_datasets = train_dataset.map(tokenize_and_align_labels, batched=True)
test_tokenized_datasets = test_dataset.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/2047 [00:00<?, ?ba/s]

  0%|          | 0/30 [00:00<?, ?ba/s]

In [19]:
data_collator = DataCollatorForTokenClassification(tokenizer)
metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [[label_list[p] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    true_labels = [[label_list[l] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"]
    }
    
trainer = Trainer(
    model,
    args,
    train_dataset=train_tokenized_datasets,
    eval_dataset=test_tokenized_datasets,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [20]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `RobertaForTokenClassification.forward` and have been ignored: tokens, tags. If tokens, tags are not expected by `RobertaForTokenClassification.forward`,  you can safely ignore this message.
/opt/conda/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2046939
  Num Epochs = 3
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 512
  Gradient Accumulation steps = 1
  Total optimization steps = 11994
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: firefish (use `wandb login --relog

/opt/conda/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
500,0.107000,0.058242,0.823735,0.699652,0.756640,0.970153
1000,0.076300,0.052231,0.828088,0.749265,0.786707,0.973056
1500,0.071900,0.048736,0.830833,0.774195,0.801515,0.974571
2000,0.069000,0.047324,0.824775,0.791832,0.807968,0.975039
2500,0.067400,0.046483,0.830424,0.799641,0.814742,0.975884
3000,0.066000,0.045003,0.838634,0.790553,0.813884,0.976022
3500,0.065000,0.044327,0.843285,0.792932,0.817333,0.976495
4000,0.064100,0.043444,0.847701,0.796342,0.821219,0.977006
4500,0.061800,0.043404,0.833055,0.818176,0.825548,0.977068
5000,0.061300,0.043141,0.844168,0.807988,0.825682,0.977375


The following columns in the evaluation set  don't have a corresponding argument in `RobertaForTokenClassification.forward` and have been ignored: tokens, tags. If tokens, tags are not expected by `RobertaForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 30000
  Batch size = 128
Saving model checkpoint to comma-distilroberta-base/checkpoint-500
Configuration saved in comma-distilroberta-base/checkpoint-500/config.json
Model weights saved in comma-distilroberta-base/checkpoint-500/pytorch_model.bin
tokenizer config file saved in comma-distilroberta-base/checkpoint-500/tokenizer_config.json
Special tokens file saved in comma-distilroberta-base/checkpoint-500/special_tokens_map.json
/opt/conda/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather

TrainOutput(global_step=11994, training_loss=0.0636678817810089, metrics={'train_runtime': 3937.2286, 'train_samples_per_second': 1559.68, 'train_steps_per_second': 3.046, 'total_flos': 2.609474674804059e+17, 'train_loss': 0.0636678817810089, 'epoch': 3.0})

In [21]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `RobertaForTokenClassification.forward` and have been ignored: tokens, tags. If tokens, tags are not expected by `RobertaForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 30000
  Batch size = 128


{'eval_loss': 0.041354574263095856,
 'eval_precision': 0.846249942164438,
 'eval_recall': 0.8208235162122742,
 'eval_f1': 0.8333428255743754,
 'eval_accuracy': 0.978227516573784,
 'eval_runtime': 27.9188,
 'eval_samples_per_second': 1074.543,
 'eval_steps_per_second': 2.113,
 'epoch': 3.0}

In [22]:
trainer.save_model("comma-" + model_checkpoint + "-3domains")

Saving model checkpoint to comma-distilroberta-base-3domains
Configuration saved in comma-distilroberta-base-3domains/config.json
Model weights saved in comma-distilroberta-base-3domains/pytorch_model.bin
tokenizer config file saved in comma-distilroberta-base-3domains/tokenizer_config.json
Special tokens file saved in comma-distilroberta-base-3domains/special_tokens_map.json
